## Modify icedraft getz shelf (make similar to Grenoble version)

In [11]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cmocean

In [2]:
base_dir = '/gws/nopw/j04/anthrofail/birgal/NEMO_AIS/'

In [59]:
my_domcfg  = xr.open_dataset(f'{base_dir}bathymetry/domain_cfg-20240305.nc').squeeze()
IGE_domcfg = xr.open_dataset(f'{base_dir}eANT025-orig/domain_cfg_eANT025.L121.nc').squeeze() 
new_domcfg = my_domcfg.copy()

In [68]:
# simple test domain_cfg:
getz_region   = np.zeros(my_domcfg.isf_draft.isel(y=slice(0,439)).values.shape)
getz_region[200:260,605:689] = 1

for var in ['isf_draft', 'top_level']:
    new_var = xr.where(getz_region==1, IGE_domcfg[var], my_domcfg[var].isel(y=slice(0,439)))
    new_var = xr.concat([new_var, my_domcfg[var].isel(y=slice(439,None))], dim='y')
    
    new_domcfg[var] = new_var

In [71]:
new_domcfg.to_netcdf(f'{base_dir}bathymetry/domain_cfg-20240305-deep-getz.nc')